In [1]:
import numpy as np
import pickle
import lzma

import elephant.statistics as estats
import elephant
import neo
from quantities import ms, s, Hz

from elephant.spike_train_dissimilarity import victor_purpura_distance
from elephant.spike_train_dissimilarity import van_rossum_distance

import pandas as pd
import matplotlib.pyplot as plt

import multiprocessing



In [2]:
def get_data(networktype, layercount, meandelay, stddelay, model_id, input_idx, n_run, CellType = "point"):
    name = networktype + '_' + str(CellType) + '_layercount' + str(layercount) + '_model' + str(model_id) + '_input' + str(input_idx) + '_stddelay' + str(stddelay) + '_meandelay' + str(meandelay) + '_nrun' + str(n_run)
    with lzma.open("./savedoutput/" + name + ".xz", "rb") as fp:
        outsaved = pickle.load(fp)
    return outsaved

def save_vpdist(output_tosave, networktype, layercount, meandelay, stddelay, model_id, input_idx, n_run, CellType = "point"):
    name = networktype + '_' + str(CellType) + '_layercount' + str(layercount) + '_model' + str(model_id) + '_input' + str(input_idx) + '_stddelay' + str(stddelay) + '_meandelay' + str(meandelay) + '_nrun' + str(n_run)
    with lzma.open("./VP_every_test/" + name + "_vp.xz", "wb") as fp:
        pickle.dump(output_tosave, fp)
        
def save_vrdist(output_tosave, networktype, layercount, meandelay, stddelay, model_id, input_idx, n_run, CellType = "point"):
    name = networktype + '_' + str(CellType) + '_layercount' + str(layercount) + '_model' + str(model_id) + '_input' + str(input_idx) + '_stddelay' + str(stddelay) + '_meandelay' + str(meandelay) + '_nrun' + str(n_run)
    with lzma.open("./VR_every_test/" + name + "_vr.xz", "wb") as fp:
        pickle.dump(output_tosave, fp)

In [3]:
def get_vpandcount(baseline, distrubed, qnum):
    # Calculating the victor purpora distace of a single cell spiking
    duration = 10000
    train1 = neo.SpikeTrain(baseline * ms, t_stop=duration*ms)
    train2 = neo.SpikeTrain(distrubed * ms, t_stop=duration*ms)
    
    # q = 1.0 / (10.0 * ms) # used in other paper
    q = qnum / ms # cost factor for shifting spikes in the victor purpura distance
    vp_dist = victor_purpura_distance([train1, train2], q)[0, 1]
    
    if len(baseline) == 0:
        return 0,0
    
    return vp_dist/len(baseline), len(baseline)

def get_vrandcount(baseline, distrubed, taunum):
    # Calculating the van rossum distace of a single cell spiking
    duration = 10000
    train1 = neo.SpikeTrain(baseline * ms, t_stop=duration*ms)
    train2 = neo.SpikeTrain(distrubed * ms, t_stop=duration*ms)
    # print('train1: ', train1)
    # print('train2: ', train2)
    
    # tau = 1s default? 10ms   why 10ms, vab rissom paper
    # tau = 10.0 * ms # time constant for the van rossum distance
    tau = 1000 * taunum * ms
    outofvrdist = van_rossum_distance([train1, train2], tau)
    vr_dist = outofvrdist[0, 1]
    
    if len(baseline) == 0:
        return 0,0
    output1 = vr_dist/ len(baseline)
    output2 = len(baseline)
    print(outofvrdist)
    return output1, output2

In [4]:
def get_vpdist_singlerun(baseline_all, distrubed_all, qnum, stddelay, layercount):
    # Getting the vp metrics between layers of network of a single simulation
    vplst_bylayer = []
    spikelst_bylayer = []
    layernumberlst_bylayer = []
    for layer_num in range(10):
        for cell_num in range(layercount):
            baseline = baseline_all[layer_num * layercount + cell_num]
            distrubed = distrubed_all[layer_num * layercount + cell_num]
            cell_vp, cell_spike_count = get_vpandcount(baseline, distrubed, qnum)
            vplst_bylayer.append(cell_vp)
            spikelst_bylayer.append(cell_spike_count)
            layernumberlst_bylayer.append(layer_num)
    # print(vplst_bylayer)
    # print(spikelst_bylayer)
    # print(layernumberlst_bylayer)
    return np.array([vplst_bylayer, spikelst_bylayer, layernumberlst_bylayer]).T

def get_vrdist_singlerun(baseline_all, distrubed_all, taunum, stddelay, layercount):
    # Getting the vr metrics between layers of network of a single simulation
    vrlst_bylayer = []
    spikelst_bylayer = []
    layernumberlst_bylayer = []
    for layer_num in range(10):
        for cell_num in range(layercount):
            baseline = baseline_all[layer_num * layercount + cell_num]
            distrubed = distrubed_all[layer_num * layercount + cell_num]
            cell_vr, cell_spike_count = get_vrandcount(baseline, distrubed, taunum)
            vrlst_bylayer.append(cell_vr)
            print(cell_vr)
            spikelst_bylayer.append(cell_spike_count)
            layernumberlst_bylayer.append(layer_num)
    return np.array([vrlst_bylayer, spikelst_bylayer, layernumberlst_bylayer]).T

In [5]:
def cal_VPdistbyStd(networktype, layercount, meandelay, stddelay, qnum):
    all_vp = []
    for model_id in np.arange(0, 10, 1):
        for input_idx in range(10):
            baseline_all = get_data(networktype, layercount, meandelay, 0.0, model_id, input_idx, 0, CellType = "point")
            for n_run in np.arange(0, 10, 1): 
                distrubed_all = get_data(networktype, layercount, meandelay, stddelay, model_id, input_idx, n_run, CellType = "point")
                singlerun_vpdist = get_vpdist_singlerun(baseline_all, distrubed_all, qnum, stddelay, layercount)
                save_vpdist(singlerun_vpdist,networktype, layercount, meandelay, stddelay, model_id, input_idx, n_run, CellType = "point") 
    return 0


# def cal_VRdistbyStd(networktype, layercount, meandelay, stddelay, taunum):
#     all_vp = []
#     for model_id in np.arange(0, 10, 1):
#         for input_idx in range(10):
#             baseline_all = get_data(networktype, layercount, meandelay, 0.0, model_id, input_idx, 0, CellType = "point")
#             for n_run in np.arange(0, 10, 1): 
#                 distrubed_all = get_data(networktype, layercount, meandelay, stddelay, model_id, input_idx, n_run, CellType = "point")
#                 singlerun_vrdist = get_vrdist_singlerun(baseline_all, distrubed_all, taunum, stddelay, layercount)
#                 save_vrdist(singlerun_vrdist,networktype, layercount, meandelay, stddelay, model_id, input_idx, n_run, CellType = "point") 
#     return 0
def cal_VRdistbyStd(networktype, layercount, meandelay, stddelay, taunum):
    counter = 0
    all_vp = []
    for model_id in np.arange(0, 10, 1):
        for input_idx in range(10):
            baseline_all = get_data(networktype, layercount, meandelay, 0.0, model_id, input_idx, 0, CellType = "point")
            for n_run in np.arange(0, 10, 1): 
                if counter == 5:
                    distrubed_all = get_data(networktype, layercount, meandelay, stddelay, model_id, input_idx, n_run, CellType = "point")
                    # print('baseline:', baseline_all)
                    # print('disturbed:', distrubed_all)
                    global debug_baseline 
                    debug_baseline = baseline_all
                    global debug_disturbed
                    debug_disturbed= distrubed_all
                    # singlerun_vrdist = get_vrdist_singlerun(baseline_all, distrubed_all, taunum, stddelay, layercount)
                    # save_vrdist(singlerun_vrdist,networktype, layercount, meandelay, stddelay, model_id, input_idx, n_run, CellType = "point")

                counter += 1
    return 0

In [6]:
def cal_VPdistbyStd_dicinput(input_dic):
    output = cal_VPdistbyStd(input_dic['networktype'], input_dic['layercount'], input_dic['meandelay'], input_dic['stddelay'], input_dic['qnum'])
    # with open('./VP_counter/VP_' + str(input_dic['counter']) + '.pkl', 'wb') as f: pickle.dump([], f)
    return 0


def cal_VRdistbyStd_dicinput(input_dic):
    output = cal_VRdistbyStd(input_dic['networktype'], input_dic['layercount'], input_dic['meandelay'], input_dic['stddelay'], input_dic['taunum'])
    # with open('./VR_counter/VP_' + str(input_dic['counter']) + '.pkl', 'wb') as f: pickle.dump([], f)
    return 0


In [7]:
argdict_lst_vp = []
argdict_lst_vr = []
i = 0
for layercount in [30, 40, 50, 60]:
    for MeanDelay_noround in np.arange(2, 3.01, 0.2):
        MeanDelay = np.round(MeanDelay_noround,1)
        for stdDelay_noround in np.arange(0, 1.01, 0.05):
            stdDelay = np.round(stdDelay_noround,2)
            argdict_vp = {'networktype':'FeedForward', 'layercount':layercount , 'meandelay':MeanDelay, 'stddelay':stdDelay, 'qnum':0.1, 'counter':i}
            argdict_lst_vp.append(argdict_vp)
            argdict_vr = {'networktype':'FeedForward', 'layercount':layercount , 'meandelay':MeanDelay, 'stddelay':stdDelay, 'taunum':0.01, 'counter':i}
            argdict_lst_vr.append(argdict_vr)
            # cal_VPdistbyStd('FeedForward', layercount, MeanDelay, stdDelay, 0.1)
            # cal_VRdistbyStd('FeedForward', layercount, MeanDelay, stdDelay, 0.01)
            # cal_VPdistbyStd_dicinput(argdict_vp)
            # cal_VRdistbyStd_dicinput(argdict_vr)
            i += 1
                

In [8]:
argdict_lst_vr[100]

{'networktype': 'FeedForward',
 'layercount': 30,
 'meandelay': 2.8,
 'stddelay': 0.8,
 'taunum': 0.01,
 'counter': 100}

In [9]:
cal_VRdistbyStd_dicinput(argdict_lst_vr[100])

0

In [ ]:
get_vrdist_singlerun(debug_baseline, debug_disturbed, 0.01, 0.8, 30)

In [12]:
get_vrandcount(debug_baseline[5], debug_disturbed[5], 0.01)

[[0.+0.00000000e+00j 0.+3.37174788e-07j]
 [0.+3.37174788e-07j 0.+0.00000000e+00j]]


(9.716852682626866e-10j, 347)

In [13]:
from neo.core import SpikeTrain
from elephant.spike_train_dissimilarity import van_rossum_distance
st_a = SpikeTrain(debug_baseline[5], units='ms', t_stop= 100000.0)
st_b = SpikeTrain(debug_baseline[5], units='ms', t_stop= 100000.0)
tau = 1000 * 0.01 * ms
vr = van_rossum_distance([st_a, st_b], tau)[0, 1]
vr

3.371747880871523e-07j

In [14]:
from neo.core import SpikeTrain
from elephant.spike_train_dissimilarity import van_rossum_distance
st_a = SpikeTrain(debug_baseline[5], units='ms', t_stop= 100000.0)
st_b = SpikeTrain(debug_disturbed[5], units='ms', t_stop= 100000.0)
tau = 1000 * 0.01 * ms
vr = van_rossum_distance([st_a, st_b], tau)[0, 1]
vr

3.371747880871523e-07j

In [15]:
1e-3j

0.001j

In [16]:
vr * 2

6.743495761743046e-07j

In [17]:
from neo.core import SpikeTrain
from elephant.spike_train_dissimilarity import van_rossum_distance
st_a = SpikeTrain(debug_baseline[5][81:]*ms, t_stop= 10000.0)
st_b = SpikeTrain(debug_baseline[5][81:]*ms, t_stop= 10000.0)
tau = 1000 * 0.01 * ms
vr = van_rossum_distance([st_a, st_b], tau)[0, 1]
vr

0.0

In [18]:
len(debug_baseline[5][81:])



266

In [24]:


import random
random.seed(72)
# Generate a sorted list of random numbers
lstexpr = sorted([random.uniform(0, 10000) for _ in range(266)])
print(lstexpr)


[116.30695832146043, 132.39600170729338, 177.8938526586882, 302.7210173367223, 313.7595457974607, 352.4106393288906, 412.0810418423071, 413.7951779908033, 549.5671428617643, 558.5560262842681, 609.427017433688, 634.6037336219912, 722.9312390602049, 734.2665480233157, 740.8545008210798, 741.2653516184275, 794.3197533729096, 832.7445364738085, 897.7654572011807, 910.7248116810107, 1004.8280224157124, 1012.2688044562889, 1016.1728100883594, 1070.9975223246727, 1114.3275664260054, 1127.1423769835064, 1143.5432887600882, 1217.4027653427765, 1256.9192752700853, 1352.8680610640542, 1375.2779916825664, 1423.055786569527, 1442.4304245192777, 1465.3525723715477, 1467.6092401453611, 1480.5528761199826, 1497.7736931073182, 1507.3314711524265, 1533.9382330692165, 1540.1859600919877, 1541.61925476618, 1609.0055864637254, 1669.5459111370726, 1726.444129921061, 1797.6886316540485, 1847.5862816044387, 1881.2858691951762, 1898.8597141561058, 1909.6439915090546, 2037.5543831823195, 2162.269197674237, 216

In [25]:
from neo.core import SpikeTrain
from elephant.spike_train_dissimilarity import van_rossum_distance
st_a = SpikeTrain(lstexpr*ms, t_stop= 10000.0)
st_b = SpikeTrain(lstexpr*ms, t_stop= 10000.0)
tau = 1000 * 0.01 * ms
vr = van_rossum_distance([st_a, st_b], tau)[0, 1]
vr

3.371747880871523e-07

In [41]:
from neo.core import SpikeTrain
from elephant.spike_train_dissimilarity import van_rossum_distance
st_a = SpikeTrain([1, 3.6, 7, 9]*ms, t_stop= 10000.0)
st_b = SpikeTrain([1, 3.6, 7, 9]*ms, t_stop= 10000.0)
tau = 1000 * 0.01 * ms
vr = van_rossum_distance([st_a, st_b], tau)[0, 1]
vr

0.0

In [15]:
from neo.core import SpikeTrain
from elephant.spike_train_dissimilarity import van_rossum_distance
st_a = SpikeTrain(debug_baseline[120]*ms, t_stop= 10000.0)
st_b = SpikeTrain(debug_disturbed[120]*ms, t_stop= 10000.0)
q = 0.1/ms
vr = victor_purpura_distance([st_a, st_b], q)[0, 1]
vr

177.84749999995822

In [38]:
from neo.core import SpikeTrain
from elephant.spike_train_dissimilarity import van_rossum_distance
st_a = SpikeTrain(debug_baseline[5]*ms, t_stop= 10000.0)
st_b = SpikeTrain(debug_disturbed[5]*ms, t_stop= 10000.0)
q = 0.1/ms
vr = victor_purpura_distance([st_a, st_b], q)[0, 1]
vr

0.0

In [37]:
debug_baseline[5]

[26.67500000009902,
 51.25000000010309,
 71.67500000010773,
 108.5000000001161,
 183.80000000006976,
 198.40000000005648,
 230.07500000002767,
 283.9999999999786,
 363.12499999990666,
 417.02499999985764,
 457.8499999998205,
 486.6499999997943,
 511.24999999977194,
 530.5499999998387,
 546.2249999998958,
 605.4250000001111,
 630.5250000002025,
 661.475000000315,
 681.9500000003895,
 699.3000000004527,
 805.6500000008396,
 838.1500000009578,
 853.875000001015,
 869.8750000010732,
 885.0500000011284,
 905.7250000012036,
 972.4500000014464,
 1000.5000000015484,
 1069.8750000018008,
 1092.775000001884,
 1110.0250000019469,
 1189.3500000022354,
 1206.5000000022978,
 1238.175000002413,
 1260.8750000024957,
 1278.4250000025595,
 1323.2000000027224,
 1363.6750000028696,
 1379.8500000029285,
 1393.8000000029792,
 1422.1250000030823,
 1442.0250000031547,
 1459.450000003218,
 1492.6500000033388,
 1547.4750000035383,
 1589.3750000036907,
 1625.475000003822,
 1677.700000004012,
 1693.4500000040694,

In [27]:
debug_disturbed[5]

[26.67500000009902,
 51.25000000010309,
 71.67500000010773,
 108.5000000001161,
 183.80000000006976,
 198.40000000005648,
 230.07500000002767,
 283.9999999999786,
 363.12499999990666,
 417.02499999985764,
 457.8499999998205,
 486.6499999997943,
 511.24999999977194,
 530.5499999998387,
 546.2249999998958,
 605.4250000001111,
 630.5250000002025,
 661.475000000315,
 681.9500000003895,
 699.3000000004527,
 805.6500000008396,
 838.1500000009578,
 853.875000001015,
 869.8750000010732,
 885.0500000011284,
 905.7250000012036,
 972.4500000014464,
 1000.5000000015484,
 1069.8750000018008,
 1092.775000001884,
 1110.0250000019469,
 1189.3500000022354,
 1206.5000000022978,
 1238.175000002413,
 1260.8750000024957,
 1278.4250000025595,
 1323.2000000027224,
 1363.6750000028696,
 1379.8500000029285,
 1393.8000000029792,
 1422.1250000030823,
 1442.0250000031547,
 1459.450000003218,
 1492.6500000033388,
 1547.4750000035383,
 1589.3750000036907,
 1625.475000003822,
 1677.700000004012,
 1693.4500000040694,